In [8]:
%matplotlib inline
import numpy as np              
import pandas as pd

In [9]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)


In [10]:
#dropping to only relevant columns
loose=loose.drop(columns=['3Langs','Item','gender','course_id','class_id','version','text1_len','text2_len','text1',
                         'text2 (line breaks/extra spaces removed, spaces added to reach 60)','Judgement','Notes'])

In [11]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [12]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

NameError: name 'rmStr' is not defined

In [17]:
# Merging the datasets

#merge_ans=answer_[['anon_id','course_id']] #sets answer df up for merging 
#student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student.merge(answer_,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs=student_ans.merge(course_, on='course_id')
#.drop(columns=['age','course_id','version','tokens']) #merges the student-answers df with course df

In [21]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

# Whole Dataset

In [18]:
stu_ans_crs

,anon_id,gender,native_language,course_history,age,answer_id,question_id,course_id,version,text_len,text,tokens,level_id
0,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,150,4,117,1,299,Some people prefer eat out and some like doing...,"['Some', 'people', 'prefer', 'eat', 'out', 'an...",5
1,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,1221,97,117,1,288,My opinion is that a person does need educatio...,"['My', 'opinion', 'is', 'that', 'a', 'person',...",5
2,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,1957,189,117,1,321,There are two national rooms in the Cathedral ...,"['There', 'are', 'two', 'national', 'rooms', '...",5
3,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,2164,190,117,1,464,There are two nation rooms in the Cathedral of...,"['There', 'are', 'two', 'nation', 'rooms', 'in...",5
4,bv5,Male,Arabic,3;9;15;21;27;49;57;71;82;93;117,21.0,151,4,117,1,315,"""Not all learning takes place in the classroom...","['``', 'Not', 'all', 'learning', 'takes', 'pla...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46225,cy7,Female,Korean,NaN,NaN,47682,6066,1034,1,10,1. emphasis\n2. appropriate\n3. requires\n4. a...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46226,fp7,Female,Turkish,NaN,NaN,47823,6066,1034,1,10,1.emphasis \n2.appropriate\n3.requires\n4.anal...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46227,fq6,Male,Chinese,NaN,NaN,47824,6066,1034,1,10,1.emphasis\n2.appropriate\n3.requires\n4.analy...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46228,di6,Male,Chinese,NaN,NaN,47787,6066,1034,1,10,1 emphasis\n2 normal\n3 requires\n4 analyze\n5...,"['1', 'emphasis', '2', 'normal', '3', 'require...",3


In [19]:
sac_lvls=stu_ans_crs[['anon_id','native_language','level_id','question_id']]
sac_lvls_rkc=sac_lvls

index=0
for i in sac_lvls_rkc.native_language:
    if (i != 'Korean' and i != 'Spanish' and i != 'Chinese'):
        sac_lvls_rkc=sac_lvls_rkc.drop(index)
    index+=1
    
sac_lvls_rkc

,anon_id,native_language,level_id,question_id
13,ax4,Chinese,5,48
14,ax4,Chinese,5,97
15,ax4,Chinese,5,74
16,ax4,Chinese,5,119
17,ax4,Chinese,5,189
...,...,...,...,...
46218,cz3,Korean,3,6087
46219,cz3,Korean,3,6119
46225,cy7,Korean,3,6066
46227,fq6,Chinese,3,6066


In [22]:
# allows us to see what levels each anon_id participated in 
lvl_list=sac_lvls_rkc.groupby('anon_id').agg(n_uniq=('question_id','nunique'), lvl_nums=('question_id',get_uniques))
lvl_list=lvl_list.reset_index()

In [28]:
unique_q=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

In [29]:
unique_q

,native_language,n_uniq
0,Chinese,3708
1,Korean,3345
2,Spanish,1513
